In [1]:
%%init_spark
launcher.packages = ["harsha2010:magellan:1.0.5-s_2.11"]
launcher.conf.set("spark.ui.port", "4041")
launcher.conf.set("spark.ui.reverseProxyUrl", "https://gisui.totosearch.org")
launcher.conf.set("spark.executor.memory", "8g")
launcher.conf.set("spark.driver.memory", "8g")
launcher.conf.set("spark.executor.cores", "2")
launcher.conf.set("spark.driver.maxResultSize", "10G")
launcher.conf.set("spark.executor.pyspark.memory", "8g")

In [2]:
import org.apache.spark.sql.SQLContext
val sqlCtx = new org.apache.spark.sql.SQLContext(sc)

Intitializing Scala interpreter ...

Spark Web UI available at http://172.17.0.2:4041
SparkContext available as 'sc' (version = 2.2.3, master = local[*], app id = local-1555419454566)
SparkSession available as 'spark'


import org.apache.spark.sql.SQLContext
sqlCtx: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@52311b62


In [3]:
import magellan.{Point, Polygon}
import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.types._
import spark.implicits._

import magellan.{Point, Polygon}
import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.types._
import spark.implicits._


In [4]:
val df = sqlCtx.read.format("magellan").load("/tf/Team 6 - Project/Finding_the_Biomes/Biomes/")
val polygons = df.select("polygon","metadata")

polygons.show()
//df.select("metadata").take(100).last(0).asInstanceOf[Map[String, String]]("BIOME_NUM").trim().toDouble


+--------------------+--------------------+
|             polygon|            metadata|
+--------------------+--------------------+
|magellan.Polygon@...|Map(xcoord -> 157...|
|magellan.Polygon@...|Map(xcoord -> 157...|
|magellan.Polygon@...|Map(xcoord -> 157...|
|magellan.Polygon@...|Map(xcoord -> -12...|
|magellan.Polygon@...|Map(xcoord -> -12...|
|magellan.Polygon@...|Map(xcoord -> 127...|
|magellan.Polygon@...|Map(xcoord -> 127...|
|magellan.Polygon@...|Map(xcoord -> 127...|
|magellan.Polygon@...|Map(xcoord -> -92...|
|magellan.Polygon@...|Map(xcoord -> -92...|
|magellan.Polygon@...|Map(xcoord -> -92...|
|magellan.Polygon@...|Map(xcoord -> 26....|
|magellan.Polygon@...|Map(xcoord -> 26....|
|magellan.Polygon@...|Map(xcoord -> 82....|
|magellan.Polygon@...|Map(xcoord -> 82....|
|magellan.Polygon@...|Map(xcoord -> 70....|
|magellan.Polygon@...|Map(xcoord -> -12...|
|magellan.Polygon@...|Map(xcoord -> -12...|
|magellan.Polygon@...|Map(xcoord -> -12...|
|magellan.Polygon@...|Map(xcoord

df: org.apache.spark.sql.DataFrame = [point: point, polyline: polyline ... 3 more fields]
polygons: org.apache.spark.sql.DataFrame = [polygon: polygon, metadata: map<string,string>]


In [5]:
val points = sc.parallelize(Seq((62.09999999998624,-88.70000000000006), (2.333333, 48.866667), (1.0, -1.0))).toDF("x", "y").select(point($"x", $"y").as("point"))

points.show()

+--------------------+
|               point|
+--------------------+
|Point(62.09999999...|
|Point(2.333333, 4...|
|    Point(1.0, -1.0)|
+--------------------+



points: org.apache.spark.sql.DataFrame = [point: point]


In [6]:
points.join(polygons).where($"point" within $"polygon").
collect().
map(x => (x(0).asInstanceOf[magellan.Point].getX(), 
          x(0).asInstanceOf[magellan.Point].getY(),
          x(2).asInstanceOf[Map[String, String]]("BIOME_NAME").trim().toString,
         x(2).asInstanceOf[Map[String, String]]("BIOME_NUM").trim().toDouble,
        x(2).asInstanceOf[Map[String, String]]("ECO_BIOME_").trim().toString)).
toSeq.
toDF("Longitude", "Latitude", "BIOME_NAME", "BIOME_NUM", "BIOME_CODE").show()


+-----------------+------------------+--------------------+---------+----------+
|        Longitude|          Latitude|          BIOME_NAME|BIOME_NUM|BIOME_CODE|
+-----------------+------------------+--------------------+---------+----------+
|62.09999999998624|-88.70000000000006|                 N/A|     11.0|       N/A|
|         2.333333|         48.866667|Temperate Broadle...|      4.0|      PA04|
+-----------------+------------------+--------------------+---------+----------+



In [7]:
def extractDouble(expectedNumber: Any): Option[Double] = expectedNumber match {
    case d: Double => Some(d)
    case i: Int => Some(i.toDouble)
    case l: Long => Some(l.toDouble)
    case s: String => scala.util.Try(s.trim.toDouble).toOption
    case _ => None
  } 

//21411
for (i <- 21411 to 21411){
    var df_points = spark.read.
    format("csv").
    option("header", "true").
    load(s"/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_$i")
    
    //df_points.show()
    var grid = df_points.collect()
    
    var points_grid = sc.parallelize(grid.map(x => (extractDouble(x(0)),extractDouble(x(1)))).collect{case (Some(i), Some(j)) => (i,j)}.toSeq).toDF("Longitude", "Latitude").select(point($"Longitude", $"Latitude").as("point"))
    
    points_grid.join(polygons).where($"point" within $"polygon").
    collect().
    map(x => (x(0).asInstanceOf[magellan.Point].getX(), 
              x(0).asInstanceOf[magellan.Point].getY(),
              x(2).asInstanceOf[Map[String, String]]("BIOME_NAME").trim().toString,
             x(2).asInstanceOf[Map[String, String]]("BIOME_NUM").trim().toDouble,
            x(2).asInstanceOf[Map[String, String]]("ECO_BIOME_").trim().toString)).
    toSeq.
    toDF("Longitude", "Latitude", "BIOME_NAME", "BIOME_NUM", "BIOME_CODE").
    write.format("csv").option("header", "true").save(s"/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_$i/grid")


}
    

org.apache.spark.sql.AnalysisException:  Path does not exist: file:/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_21411;

In [ ]:
//%%python
//import pandas as pd
//data = pd.read_csv("/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/grid_simple-Copy1.csv")

//import os
//for i in range(21410):
//    os.mkdir('/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_'+str(i+1))
//    (data.iloc[i*100:(i+1)*100,:]).to_csv('/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_'+str(i+1)+'/Grid_'+str(i+1)+'.csv', index = False)



In [1]:
//%%python
//import os
//for i in range(359,360):
//    files = os.listdir('/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_'+str(i)+'/grid')
//    for file in files:
//        os.remove('/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_'+str(i)+'/grid/' + file)
//        print("File", file, "removed")
//    os.rmdir('/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_'+str(i)+'/grid/')




Intitializing Scala interpreter ...

Spark Web UI available at http://172.17.0.2:4040
SparkContext available as 'sc' (version = 2.2.3, master = local[*], app id = local-1554750825484)
SparkSession available as 'spark'


FileNotFoundError: (2, 'No such file or directory')

In [1]:
//%%python
//import os

//import pandas as pd
//data = pd.read_csv("/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/grid_simple-Copy1.csv")

//files = os.listdir('/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/')
//print(len(data)/100)

21410.54


In [9]:
//val i = 1
//var df_points = spark.read.
//    format("csv").
//    option("header", "true").
//    load(s"/tf/Team 6 - Project/Creating shapefiles/Grid shapefiles/Grid_Simple/partial/Grid_$i/grid/")
    
//df_points.show()


+-------------------+-----------------+----------+---------+----------+
|          Longitude|         Latitude|BIOME_NAME|BIOME_NUM|BIOME_CODE|
+-------------------+-----------------+----------+---------+----------+
| -112.9000000000038|-88.5000000000001|       N/A|     11.0|       N/A|
|-112.80000000000382|-88.5000000000001|       N/A|     11.0|       N/A|
|-112.70000000000385|-88.5000000000001|       N/A|     11.0|       N/A|
|-112.60000000000386|-88.5000000000001|       N/A|     11.0|       N/A|
|-112.50000000000385|-88.5000000000001|       N/A|     11.0|       N/A|
|-112.40000000000386|-88.5000000000001|       N/A|     11.0|       N/A|
|-112.30000000000385|-88.5000000000001|       N/A|     11.0|       N/A|
|-112.20000000000385|-88.5000000000001|       N/A|     11.0|       N/A|
|-112.10000000000386|-88.5000000000001|       N/A|     11.0|       N/A|
| -107.9000000000041|-88.5000000000001|       N/A|     11.0|       N/A|
| -107.8000000000041|-88.5000000000001|       N/A|     11.0|    

i: Int = 1
df_points: org.apache.spark.sql.DataFrame = [Longitude: string, Latitude: string ... 3 more fields]
